# Segmenting and Clustering Neighborhoods in Toronto

### This notebook have the answers for the three exercises in Segmenting and Clustering Neighborhoods in Toronto Assigment. 

### To view the code for parts 2 and 3, just scroll down util you reach the subtitle PART 2 and PART 3, respectively.

## PART 1 - Scraping data and creating the Dataframe

In [169]:
# Importing all the necessary libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install beautifulsoup4
from bs4 import BeautifulSoup

!pip install html5lib

In [170]:
# Retreiving the URL and creating a Beautiful soup object

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html5lib')
#print(soup.prettify())

In [171]:
# Creating the Data Frame

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## PART 2 - Scraping data and creating the Dataframe

In [172]:
# Importing all the necessary libraries

!pip install geocoder
import geocoder # import geocoder\

In [173]:
# Retriving latitudes and longitudes

latitude=[]
longitude=[]
for code in df['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    #use the below code to test if the function is working
    #print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

In [174]:
# Adding latitudes and longitudes to the Dataframe
df['Latitude'] = latitude
df['Longitude'] = longitude
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


## PART 3 - Explore and cluster the neighborhoods in Toronto

In [175]:
# Get Toronto coordinates

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude_tor = location.latitude
longitude_tor = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_tor, longitude_tor))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [176]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [177]:
# Split the dataframe to get only the Borough with Toronto in the name

toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M4E,East Toronto,The Beaches,43.67709,-79.29547
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306


In [178]:
# Create new map of Toronto with only the selected Borough
map_toronto2 = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

In [196]:
CLIENT_ID = 'KAU3FQCHZ3QJ55E5QJDCXUFBG2UV4ALKE4ILKU1CRWGTYD1K' # your Foursquare ID
CLIENT_SECRET = '54DR5NHCR0UUENS4KEPDXIJBWKTMGNH44MV3UW0HFN0MVAOP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [180]:
# Explore Neighborhoods in Toronto


#Function to get the main venues of all the slected neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #use the below line to test if the function is working
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [181]:
# Get the main venues of all the slected neighborhoods in Toronto

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [182]:
# Check toronto_venues
print(toronto_venues.shape)
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.head()

(1698, 7)
There are 228 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


In [183]:
# Analyze Each Neighborhood

# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# Print each neighborhood along with the top 5 most common venues
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.06
2              Bakery  0.05
3  Seafood Restaurant  0.05
4          Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.06
1             Bar  0.06
2     Coffee Shop  0.05
3      Restaurant  0.05
4  Sandwich Place  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.06
1  Italian Restaurant  0.06
2                Café  0.05
3                Park  0.04
4                 Bar  0.04


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.11
1             Clothing Store  0.10
2             Sandwich Place  0.05
3  Middle Eastern Restaurant  0.03
4                Pizza Place  0.03


----Christie----
                venue  freq
0                Café  0.27
1       Grocer

In [184]:
# Put data into a pandas dataframe

# Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Restaurant,Coffee Shop,Gift Shop,Sandwich Place,Lounge,Italian Restaurant,Breakfast Spot,Nightclub
2,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Café,French Restaurant,Sandwich Place,Bar,Park,Gym / Fitness Center,Intersection,Grocery Store
3,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Pizza Place,Plaza,Bubble Tea Shop,Cosmetics Shop,Furniture / Home Store,Bookstore
4,Christie,Café,Grocery Store,Italian Restaurant,Coffee Shop,Baby Store,Athletics & Sports,Candy Store,Playground,Event Space,Dumpling Restaurant


In [185]:
# Cluster Neighborhoods

# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0, n_init = 12).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

#merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,1,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Wine Shop,Italian Restaurant,Food Truck,Event Space,Electronics Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,1,Coffee Shop,Cosmetics Shop,Hotel,Clothing Store,Cocktail Bar,Italian Restaurant,Café,Seafood Restaurant,American Restaurant,Restaurant
3,M4E,East Toronto,The Beaches,43.67709,-79.29547,1,Health Food Store,Trail,Pub,Asian Restaurant,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Breakfast Spot


In [186]:
# Create map


map_clusters = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [187]:
# Examine Clusters

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,The Danforth East,0,Convenience Store,Park,Intersection,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
12,"The Danforth West, Riverdale",0,Business Service,Ice Cream Shop,Grocery Store,Bus Line,Park,Discount Store,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
22,"High Park, The Junction South",0,Sandwich Place,Park,Residential Building (Apartment / Condo),Wings Joint,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
23,North Toronto West,0,Gym Pool,Playground,Park,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
33,Rosedale,0,Park,Japanese Restaurant,Playground,Bike Trail,Wings Joint,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


In [188]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Wine Shop,Italian Restaurant,Food Truck,Event Space,Electronics Store
1,"Garden District, Ryerson",1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Ramen Restaurant
2,St. James Town,1,Coffee Shop,Cosmetics Shop,Hotel,Clothing Store,Cocktail Bar,Italian Restaurant,Café,Seafood Restaurant,American Restaurant,Restaurant
3,The Beaches,1,Health Food Store,Trail,Pub,Asian Restaurant,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
4,Berczy Park,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Breakfast Spot
5,Central Bay Street,1,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Pizza Place,Plaza,Bubble Tea Shop,Cosmetics Shop,Furniture / Home Store,Bookstore
6,Christie,1,Café,Grocery Store,Italian Restaurant,Coffee Shop,Baby Store,Athletics & Sports,Candy Store,Playground,Event Space,Dumpling Restaurant
7,"Richmond, Adelaide, King",1,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,American Restaurant,Salad Place,Steakhouse,Asian Restaurant
8,"Dufferin, Dovercourt Village",1,Grocery Store,Park,Bakery,Pet Store,Pharmacy,Gym,Pizza Place,Middle Eastern Restaurant,Café,Smoke Shop
10,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Hotel,Japanese Restaurant,Plaza,Theater,Deli / Bodega,Park,Boat or Ferry,Electronics Store,Hotel Bar


In [189]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,2,Park,Wings Joint,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


In [190]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",3,Gym,Playground,Tennis Court,Wings Joint,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [191]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,4,Home Service,Fast Food Restaurant,Wings Joint,Dry Cleaner,Fish & Chips Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [192]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,5,Photography Studio,Bus Line,Swim School,Dry Cleaner,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


## Analysis

Most of Toronto neighborhoods have very similar most common venues, as indicated by the large number of neighborhoods classified in the same cluster (cluster 1). These venues are mainly coffee shops, cafés, and bakeries.

In cluster 0, the most common venues are parks, farmers markets, and farms. These is cohorent with the fact that all of these neighborhoods are located farther from the city center than the neighborhoods in cluster 1.

All of the remaining clusters have only one neighborhoods. All these neighborhoods are also located far from the city center, as the neighborhoods in cluster 0. Parks, farmers markets, and farms are also common in these neighborhoods, but less common than in those in cluster 0. 

One interesting fact is that all neighborhoods that are not in cluster 1 have Ethiopian Restaurant among the 10 most common venues, while none of neighborhoods in cluster 1 have Ethiopian Restaurant among the 10 most common venues. This may suggest that migrants or descendants from Ethiopia live mainly in neighborhoods far from the city center.